# Tutorial: Toy Biorefinery Example

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../../../src"))
import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

## Design the technology model.

### Technology name.

Choose a unique name for the technology being modeled: **Biorefinery v1**

Also choose a unique name for the default, reference, or base-case scenario: **Bioreference**

### Dimensions and indices.

Decide which quantities will be tracked as indices in the model, and settle on their units of measurment and default values:
1.  Types of capital costs
    * Preprocessing
    * Fermentation
    * Conversion
    * Separations
    * Utilities
2.  Types of fixed cost
    * Rent
    * Insurance
3.  Inputs to the process
    * Feedstock
    * Natural gas
4.  Outputs from the process
    * Biofuel
5.  Metrics
    * Jobs (person-hours/gal biofuel)
    * Fossil GHG (kg CO2-eq/gal biofuel)
    * Total GHG (kg CO2-eq/gal biofuel)
    * MFSP (USD/gal biofuel) - minimum fuel selling price
6.  Parameters
    * Fossil GHG (kg CO2-eq/year)
    * Biogenic GHG (kg CO2-eq/year)
    * Employment (person-hours/year)
    * Preprocessing Capital Cost (USD/biorefinery)
    * Fermentation Capital Cost (USD/biorefinery)
    * Conversion Capital Cost (USD/biorefinery)
    * Separations Capital Cost (USD/biorefinery)
    * Utilities Capital Cost (USD/biorefinery)
    * Rent (USD/year)
    * Insurance (USD/year)
    * Original Scale (unitless)
    * Scaling Factor (unitless)
    * Ideal Biofuel Yield (gal/metric ton)
    * Discount Rate (unitless)
    * Depreciation Period (year)
    * Depreciation Period Utilities (year)
    * Income Tax Rate (unitless)


Note that in each category, the numeric indices for each item are numbered with integers starting from zero.

See the [model](https://tyche.live/doc-src/formulation.html) and [database](https://tyche.live/doc-src/database.html) documentation for more details.

### Create the `indices` table.

Check to see that the data file reads correctly:

In [ ]:
my_designs = ty.Designs(path = ".",
                        name = 'tutorial-biorefinery.xlsx')
my_designs.indices.reset_index("Index").sort_values(["Type", "Offset"])

### Create the `results` table.

Check to see that the data file reads correctly:

In [ ]:
my_designs.results

### Create the `designs` table.

Check to see that the data file reads correctly:

In [ ]:
my_designs.designs.reset_index(["Variable", "Index"]).sort_values(["Variable", "Index"])

### Create the `parameters` table.

Check to see that the data file reads correctly:

In [ ]:
my_designs.parameters.reset_index("Parameter").sort_values("Tranche")

## Implement the technology model.

The implementation of a technology model consists of a capital cost function, a fixed cost function, a production function, and a metrics function.

See the [model](https://tyche.live/doc-src/formulation.html) documentation for more details.

The [src/technology/](../src/technology/) folder has examples of several technology models.

#### Capital cost function.

The capital cost function takes the scale of the operations and the array of technology parameters as arguments and it returns the capital costs for the technology.

In [ ]:
def capital_cost(scale, parameter):
    """
    Capital cost function.

    Parameters
    ----------
    scale : float
      The scale of operation.
    parameter : array
      The technological parameterization.

    Returns
    -------
    @TODO update docstring
    Total capital cost for one biorefinery (USD/biorefinery)
    """
    # "original" biorefinery scale
    o_scale = parameter[10]

    # scaling factor for equipment costs
    scale_factor = parameter[11]

    pre = parameter[3] * (scale / o_scale) ** scale_factor
    fer = parameter[4] * (scale / o_scale) ** scale_factor
    con = parameter[5] * (scale / o_scale) ** scale_factor
    sep = parameter[6] * (scale / o_scale) ** scale_factor
    utl = parameter[7] * (scale / o_scale) ** scale_factor

    return np.stack([pre, fer, con, sep, utl])

Implement the capital cost function for your technology in the file [tutorial/my_technology.py](./).

#### Fixed cost function.

The fixed cost function takes the scale of the operations and the array of technology parameters as arguments and it returns the fixed costs for the technology.

In [ ]:
def fixed_cost(scale, parameter):
    """
    Fixed cost function.

    Parameters
    ----------
    scale : float [Unused]
      The scale of operation.
    parameter : array
      The technological parameterization.

    Returns
    -------
    total fixed costs for one biorefinery (USD/year)
    """
    o_scale = parameter[10]

    rnt = parameter[8] * (scale / o_scale)
    ins = parameter[9] * (scale / o_scale)

    return np.stack([rnt, ins])

Implement the fixed cost function for your technology in the file [tutorial/my_technology.py](./).

#### Production function.

The production function takes the scale of the operations, the capital costs, the lifetime, the fixed costs, and the array of technology parameters as arguments and it returns the production (outputs) for the technology.

In [ ]:
def production(scale, capital, lifetime, fixed, input, parameter):
    """
    Production function.

    Parameters
    ----------
    scale : float
      The scale of operation.
    capital : array
      Capital costs.
    lifetime : float
      Technology lifetime.
    fixed : array
      Fixed costs.
    input : array
      Input quantities.
    parameter : array
      The technological parameterization.

    Returns
    -------
    output_raw
        Ideal/theoretical production of each technology output: biofuel at
        gals/year
    """

    theor_yield = parameter[12]
    feedstock = input[0] * scale

    # theoretical biofuel yield w/out any efficiency losses
    output_raw = theor_yield * feedstock

    return np.stack([output_raw])

Implement the production function for your technology in the file [tutorial/my_technology.py](./).

#### Metric function.

The metric function takes information on costs, inputs, outputs, and parameters and it returns the metrics for the technology.

In [ ]:
def metrics(scale, capital, lifetime, fixed, input_raw,
            input, input_price, output_raw, output, cost, parameter):
    """
    Metrics function.

    Parameters
    ----------
    scale : float
      The scale of operation. Unitless
    capital : array
      Capital costs. Units: USD/biorefinery
    lifetime : float
      Technology lifetime. Units: year
    fixed : array
      Fixed costs. Units: USD/year
    input_raw : array
      Raw input quantities (before losses). Units: metric ton feedstock/year
    input : array
      Input quantities. Units: metric ton feedstock/year
    input_price : array
        Array of input prices. Various units.
    output_raw : array
      Raw output quantities (before losses). Units: gal biofuel/year
    output : array
      Output quantities. Units: gal biofuel/year
    cost : array
      Costs.
    parameter : array
      The technological parameterization. Units vary; given in comments below
    """

    # annual fossil GHG emissions, Units: kg CO2-eq/year
    ghg_foss_ann = parameter[0]
    # annual biogenic GHG emissions, Units: kg CO2-eq/year
    ghg_bio_ann  = parameter[1]
    # Annual person-hours required, Units: person-hours/year
    emp_ann      = parameter[2]
    # Preprocessing capital cost, Units: USD
    pre_cap      = parameter[3]
    # Fermentation capital cost, Units: USD
    fer_cap      = parameter[4]
    # Conversion capital cost, Units: USD
    con_cap      = parameter[5]
    # Separations capital cost, Units: USD
    sep_cap      = parameter[6]
    # Utilities capital cost, Units: USD
    utl_cap      = parameter[7]
    # Annual rent, Units: USD/year
    rnt_ann      = parameter[8]
    # Annual insurance, Units: USD/year
    ins_ann      = parameter[9]
    # Discount rate, Unitless
    dr           = parameter[13]
    # Depreciation period for all equipment except utilities, Units: years
    dp           = parameter[14]
    # Depreciation period for utilities, Units: years
    dpu          = parameter[15]
    # Income tax rate, Units: years
    tr           = parameter[16]
    # equipment lifetime
    els          = lifetime[0]

    # JOBS: person-hours/gal biofuel
    # parameter[2] units: person-hours/year
    # output units: gal biofuel/year
    emp = emp_ann / output

    # FOSSIL GHG: kg CO2-eq/gal biofuel
    # parameter[0] units: kg CO2-eq/year
    # output units: gal biofuel/year
    ghg_foss = ghg_foss_ann / output

    # TOTAL GHG: kg CO2-eq/gal biofuel
    # parameter[0] and parameter[1] units: kg CO2-eq/year
    # output units: gal biofuel/year
    ghg_tot = (ghg_foss_ann + ghg_bio_ann) / output

    # MINIMUM FUEL SELLING PRICE: USD/gal biofuel
    # total project investment, Units: USD
    # sum of all capital costs
    tpi = pre_cap + fer_cap + con_cap + sep_cap + utl_cap

    # depreciation costs, units: USD/year
    dc = (pre_cap + fer_cap + con_cap + sep_cap) / dp + utl_cap / dpu

    # operating costs, units: USD/year
    oc = input_raw[0] * input_price[0] + input_raw[1] * input_price[1] + rnt_ann + ins_ann

    # tpi discount factor, Units: unitless
    df_tpi = (dr * (1 + dr) ** els) / ((1 + dr) ** els - 1)

    # total revenue from biofuel sales, Units: USD/year
    br = ((1 - tr) * oc - tr * dc + df_tpi * tpi) / (1 - tr)

    # MFSP, Units: USD/gal biofuel
    mfsp = br / output
    
    return np.stack([emp, ghg_foss, ghg_tot, mfsp])

Implement the metrics function for your technology in the file [tutorial/my_technology.py](./).

### Checking math in the metrics function

In [ ]:
parameter_check = np.array([10.0, 10.0, 10.0,              # fossil GHG, bio GHG, employment
                            10.0, 10.0, 10.0, 10.0, 10.0,  # capital costs
                            5.0, 5.0,                      # fixed costs
                            1.0, 1.0,                      # original scale, scaling factor
                            1.0,                           # ideal biofuel yield
                            0.1,                           # discount rate
                            10.0, 10.0,                    # depreciation periods
                            0.1])                          # income tax rate
output_check = 10.0
input_raw_check = np.array([10.0, 10.0])
input_price_check=  np.array([1.0, 1.0])

metrics(scale = 1.0,
        capital = np.array([10.0, 10.0, 10.0, 10.0, 10.0]),
        lifetime = np.array([10.0, 10.0, 10.0, 10.0, 10.0]),
        fixed = np.array([5.0, 5.0]),
        input_raw = input_raw_check,
        input = input_raw_check,
        input_price = input_price_check,
        output_raw = output_check,
        output = output_check,
        cost = np.array([1.0]),
        parameter = parameter_check)

### Create the `functions` table.

Check to see that the data file reads correctly:

In [ ]:
my_designs.functions

Check to see that the functions compile without errors.

In [ ]:
my_designs.compile()

## Create investment for reference case.

### Create the `tranches` table.

Check to see that the data file reads correctly:

In [ ]:
my_investments = ty.Investments(path = ".",
                                name = 'tutorial-biorefinery.xlsx')
my_investments.tranches

### Create the `investments` table.

Check to see that the data file reads correctly:

In [ ]:
my_investments = ty.Investments(path = ".",
                                name = 'tutorial-biorefinery.xlsx')
my_investments.investments

## Test the optimization feasibility.

In [ ]:
my_designs = ty.Designs(path = ".",
                        name = 'tutorial-biorefinery.xlsx')
my_investments = ty.Investments(path = ".",
                                name = 'tutorial-biorefinery.xlsx')
my_designs.compile()
investment_results = my_investments.evaluate_investments(my_designs, sample_count=50)
tranche_results = my_investments.evaluate_tranches(my_designs, sample_count=50)

In [ ]:
evaluator = ty.Evaluator(tranche_results)

In [ ]:
_wide = evaluator.evaluate_corners_wide(np.mean).reset_index()

In [ ]:
investment_results.metrics

In [ ]:
g = sb.boxplot(
    x="Investment",
    y="Value",
    data=investment_results.metrics.xs(
        "Total GHG",
        level="Index"
    ).groupby(["Investment", "Sample"]).sum(numeric_only=True).reset_index()[["Investment", "Value"]],
    order=[
        "No R&D"   ,
        "Low R&D"  ,
        "Moderate R&D",
        "High R&D"  ,
    ]
)

Now, use the optimization methods to explore the greatest potential reduction in MFSP under a variety of budget and metric constraints.

In [ ]:
optimizer = ty.EpsilonConstraintOptimizer(evaluator)

The `optimum_metrics` method does not need to be called, but it is useful to get an idea of the best possible metric values. Use the `sense` parameter (Dictionary format) to specify whether each metric should be minimized or maximized. If no `sense` is provided, all metrics are maximized.

Note that the solutions provided by `optimum_metrics` are not likely to be the same as those found through the optimization methods. Applying budget constraints and/or metric constraints will change the optimal objective function value.

In [ ]:
q = optimizer.optimum_metrics(verbose = 0,
                             sense = {'Fossil GHG': 'min',
                                     'Jobs': 'max',
                                     'MFSP': 'min',
                                     'Total GHG': 'min'})
q

Now use `opt_slsqp` to see what the lowest possible MFSP is with an budget of $3e8 and no constraints on where the budget is spent or on other metrics.

In [ ]:
slsqp_result = optimizer.opt_slsqp(
    "MFSP"                                                       ,
    sense = 'min'                                                ,
    total_amount = 3e8                                           ,
    verbose      = 0                                             ,
)
slsqp_result[1]

In [ ]:
slsqp_result[3]

In [ ]:
slsqp_result[2]

In [ ]:
diffev_result = optimizer.opt_diffev(
    "MFSP"                                                       ,
    sense = 'min'                                                ,
    total_amount = 3e8                                           ,
    verbose      = 0                                             ,
)
diffev_result[1]

In [ ]:
diffev_result[3]

In [ ]:
diffev_result[2]

The SHGO optimizer runs quickly but can stall if the `maxiter` parameter is specified. Here we see that SHGO without `maxiter` solves quickly and provides comparable results to SLSQP.

In [ ]:
shgo_result = optimizer.opt_shgo(
    "MFSP"                                                       ,
    sense = 'min'                                                ,
    total_amount = 3e8                                           ,
    verbose      = 0                                             ,
)
shgo_result[1]

In [ ]:
shgo_result[3]

In [ ]:
shgo_result[2]

We may also be interested in trade-offs between metrics. Now we can use `opt_slsqp` with the `eps_metric` parameter to again minimize MFSP, this time under both a budget constraint and an upper bound on the Total GHG emissions metric. From the `optimum_metric` results above, we know that the lowest possible value of Total GHG emissions is 13.66. We can use this information to decide on a reasonable upper bound for the constraint.

In [ ]:
slsqp_result = optimizer.opt_slsqp(
    "MFSP"                                                        ,
    sense = 'min'                                                 ,
    eps_metric = {'Total GHG': {'limit': 13.8, 'sense': 'upper'}} ,
    total_amount = 3e8                                            ,
    verbose      = 0                                              ,
)
slsqp_result[1]

An Iteration limit reached message means that the optimizer was unable to converge to an optimal solution within the specified maximum number of iterations, which defaults to 50. We can increase the iteration limit and re-solve to see if an optimal solution can be found.

In [ ]:
slsqp_result = optimizer.opt_slsqp(
    "MFSP"                                                        ,
    sense = 'min'                                                 ,
    eps_metric = {'Total GHG': {'limit': 13.8, 'sense': 'upper'}} ,
    total_amount = 3e8                                            ,
    maxiter = 200,
    verbose      = 0                                              ,
)
slsqp_result[1]

There is still no optimal solution, although if we look at the non-converged solution, we do see some improvement in Total GHG emissions and in MFSP.

In [ ]:
slsqp_result[3]

In [ ]:
slsqp_result[2]

Another option if an optimal solution is not quickly found is to alter the budget constraint, to see if additional funds will allow the metric constraint to be met.

In [ ]:
slsqp_result = optimizer.opt_slsqp(
    "MFSP"                                                        ,
    sense = 'min'                                                 ,
    eps_metric = {'Total GHG': {'limit': 13.8, 'sense': 'upper'}} ,
    total_amount = 5e8                                            ,
    maxiter = 200,
    verbose      = 0                                              ,
)
slsqp_result[1]

In [ ]:
slsqp_result[3]

In [ ]:
slsqp_result[2]

In [ ]:
diffev_eps_result = optimizer.opt_diffev(
    "MFSP"                                                        ,
    sense = 'min'                                                 ,
    eps_metric = {'Total GHG': {'limit': 13.8, 'sense': 'upper'}} ,
    total_amount = 3e8                                            ,
    verbose      = 0                                              ,
)
diffev_eps_result[1]

In [ ]:
#shgo_eps_result = optimizer.opt_shgo(
#    "MFSP"                                                        ,
#    sense = 'min'                                                 ,
#    eps_metric = {'Total GHG': {'limit': 13.8, 'sense': 'upper'}} ,
#    total_amount = 3e8                                            ,
#    verbose      = 0                                              ,
#)
#shgo_eps_result[1]

In [ ]:
pwlinear_result = optimizer.opt_milp(
    "MFSP",
    sense='min',
    total_amount = 3e8,
    verbose = 2
)

In [ ]:
pwlinear_result.exit_message

In [ ]:
pwlinear_result.metrics

In [ ]:
pwlinear_result.amounts

In [ ]:
pwlinear_result.solve_time

In [ ]:
pwlinear_eps_result = optimizer.opt_milp(
    "MFSP",
    sense='min',
    eps_metric = {'Total GHG': {'limit': 14.0, 'sense': 'upper'}} ,
    total_amount = 3e8,
    verbose = 2
)

In [ ]:
pwlinear_eps_result.exit_message